In [ ]:
import numpy as np
from MetaLearner import *
import pickle

In [ ]:
# todo: https://stackoverflow.com/questions/24168569/how-to-add-names-to-a-numpy-array-without-changing-its-dimension

In [ ]:
# This file runs the experiment

In [ ]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [ ]:
n_setups = 2
sample_sizes = [200, 200]
n_runs = 2
learners = [TLearner('rf'), SLearner('rf'), XLearner('rf'), RLearner('rf'), DRLearner('rf'), RALearner('rf'),
            PWLearner('rf'), ULearner('rf'),
            TLearner('lasso'), SLearner('lasso'), XLearner('lasso'), RLearner('lasso'), DRLearner('lasso'),
            RALearner('lasso'), PWLearner('lasso'), ULearner('lasso'),
            TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

base_learners = ['rf', 'lasso', 'nn']


In [ ]:
# What we want: For each setting and base learner --> matrix which includes the MSE, columns: Metalearners + SampleSize, rows: Run(1-10)
# --> So we can plot the mean and 1se of the MSE for each Metalearer, each samplesize; giving the setting and baselearner
# --> number of plots: settings*baselearner.

In [ ]:
results = []

for i in range(n_setups):
    results.append([])

for i in range(n_setups):
    for baselearner in range(3):
        results[i].append([])


In [ ]:
results[27][2]  # first number indicates the setup, second number indicates the base learner.

In [ ]:
mses = []

In [ ]:
# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(data, setup, samplesize, run, train_test):
    y = data[setup][samplesize][run][train_test][:, 0]
    x = data[setup][samplesize][run][train_test][:, 1:21]
    w = data[setup][samplesize][run][train_test][:, 21]
    tau = data[setup][samplesize][run][train_test][:, 22]
    return y, x, w, tau

In [ ]:
# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(data, setup, samplesize, run, train_test):
    y = data[setup][samplesize][run][train_test][:, 0]
    x = data[setup][samplesize][run][train_test][:, 1:26]
    w = data[setup][samplesize][run][train_test][:, 26]
    tau = data[setup][samplesize][run][train_test][:, 27]
    return y, x, w, tau

In [ ]:
b = 0
m = 0
s = 0
r = 0

for i in range(n_setups):
    b = 0  # restart index
    for base in range(3):  # change to baselearner
        plot_mse = np.empty(shape=(0, 8))  #
        s = 0
        for size in sample_sizes:  # change to sample_sizes
            size_mse = np.empty(shape=(n_runs, 8))  # 10 = number of runs
            for r in range(n_runs):  # run
                mses = np.empty(shape=(1, 8))  # reset
                # set data TODO: make helper function to make y, x, w, tau, out of data[][][][]
                temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(data=data, setup=i,
                                                                                         samplesize=s, run=r,
                                                                                         train_test=0)
                temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(data=data, setup=i, samplesize=s,
                                                                                     run=r, train_test=1)
                s += 1
                m = 0
            for l in learners:  # change to metalearner
                # train and test
                learner = l  # TODO: just abstract, change to metalearner with baselearner here.
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - tau_test) ** 2).mean()
                mses[0, m] = temp_mse
                m += 1
            size_mse[r, :] = mses
        b += 1
        np.append(plot_mse, mses)
    results[i][b] = plot_mse







# this should make sense i guess! :)
# TODO: TRY IT OUT TOMORROW!!!!!!, ALSO CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.





In [ ]:
#TODO: TRY THIS ONE FIRST!!!

b = 0
m = 0
s = 0
r = 0

for i in range(n_setups):
    plot_mse = np.empty(shape=(0, 24))  # baselearner * metalearner
    s = 0
    for size in sample_sizes:  # change to sample_sizes
        size_mse = np.empty(shape=(n_runs, 24))  # 10 = number of runs
        for r in range(n_runs):  # run
            mses = np.empty(shape=(1, 24))  # reset
            # set data TODO: make helper function to make y, x, w, tau, out of data[][][][]
            temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(data=data, setup=i,
                                                                                     samplesize=s, run=r,
                                                                                     train_test=0)
            temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(data=data, setup=i, samplesize=s,
                                                                                 run=r, train_test=1)

            m = 0
            for l in learners:  # change to metalearner
                # train and test
                learner = l  # TODO: just abstract, change to metalearner with baselearner here.
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - temp_tau_test) ** 2).mean()
                mses[0, m] = temp_mse
                m += 1
            size_mse[r, :] = mses
        s += 1
        np.append(plot_mse, mses)
    results[i][0] = plot_mse[0:n_runs, :]
    results[i][1] = plot_mse[n_runs:(2 * n_runs), :]  # change
    results[i][2] = plot_mse[(2 * n_runs):(3 * n_runs), :]  # change




############# TODO: TODO: lueg nomal, überleg logisch.

# TODO: IT WORKED SOMEHOW, BUT ONLY GET 'MSES' (ONE ROW) VECTOR AND 'SIZE_MSE' (TWO ROWS)
# this should make sense i guess! :)
# TODO: TRY IT OUT TOMORROW!!!!!!, ALSO CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.
# TODO: WHILE TRAINING: ConvergenceWarning: Try increasing tol.
# TODO: WHAT DOES PLOT.MSE ???


### TODO: solve : "Input y contains infinity or a value too large for dtype('float64')."
### for: pseudo_outcomes = (y - self.mux_model.predict(x)) / (w - probs)

### IT TOOOK 5M 40S FOR --> SAMPLE SIZES [200, 200], TEST SIZE =50, 2 SETUPS, 2 RUNS LOL

In [ ]:
results[1][2]

In [ ]:
mses

In [ ]:
size_mse[0:2, :]
# they do not look good lol, a lot ot zeros, few Nans --> LOL just my fault its the empty matrix :)

In [ ]:
plot_mse

In [ ]:
eone_line = np.concatenate((np.empty(shape=(1, 8)), np.array([[1000]])), axis=1)
np.concatenate((one_line, one_line, one_line), axis=0)

In [ ]:
np.repeat([1000], 10)

In [ ]:
one_line[0, 0] = 2

In [ ]:
one_line

In [ ]:
a = ...

In [ ]:
emp = np.empty(shape=(40, 8))

In [ ]:
np.append(emp, one_line[:, -9], axis=0)

In [ ]:
emp

In [ ]:
np.append(arr=, values=one_line)